In [1]:
import pandas as pd
import re
df= pd.read_csv('fakenewstrain.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [2]:
df=df.dropna()

In [3]:
X = df.drop('label', axis=1)
y=df['label']
X.head()


,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [4]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [5]:
import tensorflow as tf

In [6]:
tf.__version__

'2.11.0'

In [7]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [8]:
voc_size = 5000

In [9]:
messages = X.copy()

In [10]:
messages.reset_index(inplace=True)

In [11]:
import nltk 
import re
from nltk.corpus import stopwords

In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rahat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus =[]

In [14]:
for i in range(0, len(messages)):
    print(i, end='\r')
    review = re.sub('[^a-zA-Z]',' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [15]:
len(corpus)

18285

In [16]:
onehot_rep = [one_hot(words, voc_size) for words in corpus]
len(onehot_rep)

18285

In [17]:
sent_length = 20
embedded_docs = pad_sequences(onehot_rep, padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2304 2542    4]
 [   0    0    0 ... 2565 4718  461]
 [   0    0    0 ...  864 4875  846]
 ...
 [   0    0    0 ... 2216 3799 4184]
 [   0    0    0 ... 1945  272 3413]
 [   0    0    0 ... 4290 2356 1827]]


In [18]:
len(embedded_docs)

18285

In [19]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
import numpy as np
X_final=np.array(embedded_docs)
y_final = np.array(y)

In [21]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [22]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X_final,y_final, test_size=0.33, random_state=43)

In [23]:
model.fit(X_train,y_train, validation_data=(X_test,y_test),epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 18ms/step - loss: 0.3273 - accuracy: 0.8504 - val_loss: 0.1881 - val_accuracy: 0.9180
Epoch 2/10
192/192 [==============================] - 3s 15ms/step - loss: 0.1369 - accuracy: 0.9456 - val_loss: 0.1849 - val_accuracy: 0.9221
Epoch 3/10
192/192 [==============================] - 3s 16ms/step - loss: 0.0933 - accuracy: 0.9661 - val_loss: 0.2134 - val_accuracy: 0.9067
Epoch 4/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0680 - accuracy: 0.9771 - val_loss: 0.2202 - val_accuracy: 0.9158
Epoch 5/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0415 - accuracy: 0.9866 - val_loss: 0.2628 - val_accuracy: 0.9158
Epoch 6/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0246 - accuracy: 0.9924 - val_loss: 0.3190 - val_accuracy: 0.9157
Epoch 7/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0167 - accuracy: 0.9945 - val_loss: 0.3591 - val_accuracy: 0.9094

In [24]:
from tensorflow.keras.layers import Dropout

embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [31]:
y_pred= model.predict_classes(X_test)
y_pred

AttributeError: 'Sequential' object has no attribute 'predict_classes'

In [32]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

ValueError: Classification metrics can't handle a mix of binary and continuous targets